In [18]:
from bs4 import BeautifulSoup as bs
import time
import sqlite3 
import re
from datetime import datetime
from selenium import webdriver
import json
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import telebot
import threading

# Start all functions

In [19]:
# client_chat = 779224126
# client_companies = '[2314,4526,124]'

def add_client(client_chat,client_companies):
    """ Adding new client to the database"""

    conn = sqlite3.connect('Interfax_bot.sqlite')

    cursor = conn.cursor()

    # Данные нужно передавать кортежем. Даже одну запись
    
    client_companies = create_string(client_companies)
    
    cursor.execute('select ID from Clients where Client_name = ?;',(client_chat,))
    client = cursor.fetchone()
    
    if client == None:
        cursor.execute("insert into Clients values (Null, ?, ?);", (client_chat, client_companies))
    else:
        cursor.execute('''update Clients 
                          set Client_name = ?, companies = ? 
                          where ID = ?;''',(client_chat, client_companies, client[0]))

    conn.commit()

# Использует функцию выше внутри себя. Проверяет наличие клиента в базе и добавляет его/обновляет его подписки
def add_update(name,companies):    
    
    conn = sqlite3.connect('Interfax_bot.sqlite')

    cursor = conn.cursor()

    cursor.execute('select ID from Clients where Client_name = ?;',(name,))

    res = cursor.fetchone()

    if res == None:
        add_client(name,companies)
    else:
        cursor.execute('''update Clients 
                          set Client_name = ?, companies = ? 
                          where ID = ?;''',(name, companies, res[0]))
        conn.commit() 
    # res[0] is the ID value itself, since fetchone returns a tuple


# эти две функции используются,чтобы хранить данные по клиентам в таблице в виде строки
def retrieve_code(string):
    dic = list(map(str, string.replace('{','').replace('}','').split(',')))
    dic = set(dic)
    return dic

def create_string(array):
    
    string = '{'
    for i in array:
        if i == '':
            continue
        string = string + str(i) + ','
    string = string[:-1] + '}'
    return string


def add_subscribers(companies,client):
    """ Updates the set of subscribers for every company """
    

    conn = sqlite3.connect('Company_codes_to_clients.sqlite')
    cursor = conn.cursor()
    
#   Since this module will work right after the client chooses companies, the given array of companies codes will be a set
    for i in companies:

    # retrieve the existing string of clients for the company
    # convert it to set
    # add the client to this set
    # convert back to string
    # update the table

        cursor.execute('select Chat_codes from Companies where Company_name = ?;',(i,))
        chat_codes = retrieve_code((cursor.fetchone()[0]))
        chat_codes.add(client)
        update = create_string(chat_codes)
        cursor.execute("""
                        update Companies 
                        set Chat_codes = ?
                        where Company_name = ?

                        """,(update,i))
        conn.commit()

        
def subscribers(link,info):
    """ sends news to all subscribers of firms from the latest update"""
    conn = sqlite3.connect('Company_codes_to_clients.sqlite')
    
    cursor = conn.cursor()
    code = firm_code_from_link(link)
    
    cursor.execute('select Chat_codes from Companies where Code = ?', (code,))
    chat_codes = cursor.fetchone()
    
    
    if chat_codes == None:
        pass
    elif chat_codes[0] == "{}":
        pass
    else:
        for i in retrieve_code(chat_codes[0]):
            bot.send_message(i,info)
            
        
        
        
def firm_code_from_link(link):
    """ retrieves company code from link"""
    equal = link.rfind('=')
    code = link[equal+1:]
    return code




# Функция поиска и обновления новостей. 

td_last = 250620201900
def find_send_news():
    while True:
        url = 'https://www.e-disclosure.ru/poisk-po-soobshheniyam'

        driver = webdriver.Firefox()
        driver.implicitly_wait(10)
        driver.get(url)

        python_button = driver.find_element_by_id('butt') #FHSU
        python_button.click() #click fhsu link
        #  Выбрать пункт 10 и нажать
        python_button2 = driver.find_element_by_xpath('//*[@id="pageSize"]/option[1]')
        python_button2.click()

        time.sleep(3)
        soup = bs(driver.page_source, 'lxml')
        res = soup.find_all('tbody')

        table = res[-9]

        driver.quit()


        global td_last

        #  Here i need to define all the variables I will use in this module
        def number_date(text):
            '''Function to convert date into an int for comparison and searching relevant news'''
            pre_res = text.split()
            one = pre_res[0].replace('.','') 
            two = pre_res[1].replace(':','')
            res = int(one+two)
            return res



        iterat = table.contents  # all tr in the table

        for child in range(0,len(iterat),2):
        # for some reason there are empty cells in [0],[2],[4] and etc. rows, so I need to go through odd ones
            date = iterat[child].contents[1].text # first td with the date and time
            if number_date(date) > td_last:
        #         iterat[child].contents[3] is the second td containing company name and the event name. However, there are two <a> tags, 
        #         one with company name, the other with the event. Therefore another method, contents[0].text - company and contents[3].text - event, is used.
                info = str(iterat[child].contents[3].contents[0].text) + '\n' + str(iterat[child].contents[3].contents[3].text) + ' ' + str(iterat[child].contents[3].contents[3].get('href'))
                link = iterat[child].contents[3].contents[0].get('href') 


                subscribers(link, info)
            # td_last is the last time we checked news - it is equiualent to the top row news data in the current session.



        td_last = number_date(iterat[0].contents[1].text)   
        time.sleep(5)

### The Bot

In [27]:
token = ''
bot = telebot.TeleBot(token)

keyboard1 = telebot.types.ReplyKeyboardMarkup()
keyboard1.row('Последние новости', 'Выбрать компании')

keyboard2 = telebot.types.InlineKeyboardMarkup(); #наша клавиатура
# key_one = telebot.types.InlineKeyboardButton(text='Новости', callback_data='news'); #кнопка «Да»
# keyboard2.add(key_one); #добавляем кнопку в клавиатуру
key_two = telebot.types.InlineKeyboardButton(text='Выбрать компании', callback_data='companies');
keyboard2.add(key_two);


@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, я бот раскрытия. В текущей версии я буду отправлять тебе новости о последних раскрытиях в реальном времени.')
    bot.send_message(message.chat.id, 'Какую опцию выберете?', reply_markup = keyboard2)
    chat_id = message.chat.id
#     I need to add clients chat_id to SQLite database later

@bot.message_handler(content_types = ['text'])

def check_client(message):
    
    # Словарь для хранения компаний каждого клиента. Он будет вызываться в функции добавления компаний
    
    
    conn = sqlite3.connect('Interfax_bot.sqlite')

    cursor = conn.cursor()

    cursor.execute('select companies from Clients where Client_name = ?;',(message.chat.id,))

    try:
        pre_res = cursor.fetchone()[0]
        
        res = retrieve_code(pre_res)
        
        global client_companies
        client_companies[message.chat.id] = res
        
        bot.send_message(message.chat.id,'У вас уже есть подписка на следующие компании')
        for i in res:
            bot.send_message(message.chat.id, str(i))

#         bot.send_message(message.chat.id, 'Хотите отписаться от некоторых компаний или добавить новые?')
        msg = bot.send_message(message.chat.id,'Напишите название компании если хотите добавить еще')
        bot.register_next_step_handler(msg,choose_companies)

    except:
        msg = bot.send_message(message.chat.id,'Напишите название компании ')
        bot.register_next_step_handler(msg,choose_companies)

def init_choose_companies(message):


    conn = sqlite3.connect('Company_codes_to_clients.sqlite')
    cursor = conn.cursor()
    cursor.execute('select Company_name from Companies')

    comp = cursor.fetchall()

    global companies

    companies = list()

    for i in comp:
        companies.append(i[0]) 


    check_client(message)

#     msg = bot.send_message(message.chat.id,'Напишите название компании')
#     bot.register_next_step_handler(msg,choose_companies)





        
try:
    comp_list = client_companies[message.chat.id]    
except:
    comp_list = set()

def  choose_companies(message):

    
    company = message.text

    if company =='/end':


# Добавляю в базу компании - клиенты
        add_subscribers(comp_list, message.chat.id)
# Добавляю в базу клиенты - компании
        add_client(message.chat.id,comp_list)

        bot.send_message(message.chat.id,'Спасибо. Подписка оформлена')
#         here I nullify the variable comp_list for the next user - need to check whether this works

    else:
        
        
        comp_results = list()
        for i in companies:
            if company.lower() in i:
                comp_results.append(i)

        if len(comp_results) == 1:
            bot.send_message(message.chat.id, 'Добавил')
            comp_list.add(comp_results[0])
            msg = bot.send_message(message.chat.id, 'Напишите название следующей компании или /end если хотите закончить.')
            bot.register_next_step_handler(msg, choose_companies)

        elif len(comp_results) > 1:
            msg = bot.send_message(message.chat.id, 'Я нашел несколько результатов. Напишите название того, который нужен')

            for a in comp_results:
                bot.send_message(message.chat.id, str(a))

            bot.register_next_step_handler(msg, choose_companies)

        else:
            msg = bot.send_message(message.chat.id, 'Не смог найти такую компанию.' +'\n' + 'Напишите название следующей компании или /end если хотите закончить.')
            bot.register_next_step_handler(msg, choose_companies)    



@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    chat = call.message
#     if call.data == "news": #call.data это callback_data, которую мы указали при объявлении кнопки
#         find_news()
#         send_news()
    if call.data == "companies":
        init_choose_companies(chat)


# Добавил многопоточность для одновременной работы бота и обновления новостей 
t1 = threading.Thread(target = find_send_news)  
t1.start() 
time.sleep(5)
t2 = threading.Thread(target = bot.polling()) 
t2.start()   



# Finished Modules

## Companies_List - use when you need to update the list

In [19]:
# url = 'https://www.e-disclosure.ru/poisk-po-soobshheniyam'

# driver = webdriver.Firefox()
# driver.implicitly_wait(10)
# driver.get(url)

# python_button = driver.find_element_by_xpath('//*[@id="selected_type"]/span') #Выбор меню сообщений
# python_button.click() #click link

# #Нажатие кнопки плюса по акциям
# python_button2 = driver.find_element_by_xpath('//*[@id="EventTypesTree"]/li[3]/div') 
# python_button2.click()

# # Выбираю все 20 пунктов выпадающего меню
# for i in range(1,20):
#     python_button3 = driver.find_element_by_xpath('//*[@id="EventTypesTree"]/li[3]/ul/li['+ str(i) +']/div/table/tbody/tr/td[2]/input')
#     python_button3.click()

    
# #Нажатие кнопки плюса по важным событиям   
# button2_2 = driver.find_element_by_xpath('//*[@id="EventTypesTree"]/li[15]/div')
# button2_2.click()

# for i in range(1,6):
#     button3_2 = driver.find_element_by_xpath('//*[@id="EventTypesTree"]/li[15]/ul/li['+ str(i) +']/div/table/tbody/tr/td[2]/input')
#     button3_2.click()
    
# # Закрываю текущее меню
# button_close = driver.find_element_by_xpath('//*[@id="type"]/table/tbody/tr[4]/td[2]/table/tbody/tr/td[1]/a')
# button_close.click()

# time.sleep(3)

# button_search = driver.find_element_by_xpath('//*[@id="butt"]')
# button_search.click()

# time.sleep(3)

# # select 1200 last updates
# python_button1200 = driver.find_element_by_xpath('//*[@id="pageSize"]/option[3]')
# python_button1200.click()

# time.sleep(2)

# soup = bs(driver.page_source, 'lxml')
# res = soup.find_all('a')
# driver.quit()

# companies = {}
# for a  in range(46,2445,2):
# #     remove comma only when need to create a dict
#     companies[res[a].text] = res[a].get('href')[-5:]



# for i in companies.keys():
#     if companies[i].startswith('='):
#         companies[i] = companies[i].replace('=','')

#     if companies[i].startswith('d='):
#         companies[i] = companies[i].replace('d=','')
# companies

In [22]:
# company = list(companies.keys())
# comp_id = list(companies.values())

# data = {'company':company, 'comp_id':comp_id}
# df = pd.DataFrame.from_dict(data)
# df

,company,comp_id
0,"АО ""ЗСД""",26349
1,ПАО “Совкомфлот”,11967
2,"ПАО ""Уралкалий""",1233
3,ПАО «Тучковский КСМ»,7779
4,"ПАО ""ММЦБ""",37646
...,...,...
621,"ОАО ""ЛИИП им. Гризодубовой В. С.",14084
622,"ПАО ""Таганрогбанк""",2696
623,"ПАО ИФ ""Аз-Капитал""",167
624,"ОАО ""Директ-фонд""",18643


In [24]:
# df.to_csv('interfax_companies_2.csv', encoding = 'windows - 1251')

# Add the companies to the list

In [34]:
# I need to read it from Sqlite and not keep in memory, since the users will not add new companies constantly
# # В текущей версии список всех компаний находится в 2 разных таблицах
# companies_names = pd.read_csv('interfax_companies.csv', encoding = 'windows-1251', index_col = 0)
# companies_names_2 = pd.read_csv('interfax_companies_2.csv', encoding = 'windows-1251', index_col = 0)

# cumulative_table = pd.concat([companies_names,companies_names_2], axis = 0).reset_index()
# cumulative_table.drop('index', axis = 1)
# companies = companies_names['company'].to_list()
# ids = companies_names['comp_id'].to_list()

# dictio = {}
# companies_low = [i.lower() for i in companies]

# for i in range(len(companies)):
#     dictio[companies_low[i]] = ids[i]
# print(dictio)


# # UNCOMMENT THIS WHEN NEED TO UPDATE THE DATABASE - BUT BE CAREFUL. IT WILL REMOVE ALL THE EXISTING CLIENTS FOR EACH COMPANY
# # SO CREATE THE DICTION WITH ONLY NEW COMPANIES AND DO THAT

# conn = sqlite3.connect('Company_codes_to_clients.sqlite')

# cursor = conn.cursor()


# for i in range(len(companies)):
#     cursor.execute('insert into Companies values (Null, ?,?,?)',(companies_low[i],ids[i],'{}'))
# conn.commit()

{'ао "дом.рф"': '1263', 'пао банк зенит': '538', 'пао "калужская сбытовая компания"': '5830', 'пао мособлбанк': '26831', 'оао "левенгук"': '31319', 'пао "гк "нижегородский"': '9613', 'пао "кзф"': '888', 'ао бпжт': '20410', 'оао "швейная фабрика "победа"': '32716', 'ао "пз "расцвет"': '36290', 'оао "мповти"': '2947', 'ао "авангард-агро"': '16644', 'ооо «сфо сфи»': '37574', 'оао "демидов-авто"': '20089', 'ао "ифз"': '4308', 'оао "дмз"': '1068', 'ао "промтрактор"': '5653', 'ао "торговый дом казанский цум"': '5305', 'оао "соз"': '3396', 'ао "бываловский машиностроительный завод"': '19932', 'пао "воэ" пао "волгоградоблэлектро"': '11286', 'ооо "лента"': '32010', 'оао "рорз"': '32774', 'ао  "мехколонна"': '10211', 'ао «смп банк»': '26869', 'ао "мособлстрой № 5"': '11789', 'аор "туринский цбз"': '9542', 'псф "автодизель-сервис" ао': '19818', 'ооо "каркаде"': '11584', 'ао "софтлайн трейд"': '37065', 'ао «сбербанк управление активами»': '6731', 'ао "элион"': '14187', 'зао «иа «московский»': '342

# Check updates for specific Firms - frozen

In [ ]:
# dictio = {}
# dictio['ПАО Сбербанк'] = 3043
# name = dictio.keys()





# def search_by_companies(*companies):
#     for company in companies:
# #         create the dict to control the number of new events
#         company_pre = {company:0}
#         request = urlopen('https://www.e-disclosure.ru/Event/Page?companyId=' + str(dictio[company]) +'&year=2020')
#         soup = bs(request, 'lxml')

        
#         # It is supposed to infinitely parse ---> don't need to check more than 20 first values
#         result = soup.find_all('td')[:15]

#         if len(result) > company_pre[company]:
#             i = 1
#             for td in range ((len(result) - result_pre)//3):

#                     print(name, 'Дата публикации - ' + str(result[i].text),result[i+1].text,result[i+1].a.get('href'),sep = '\n')
#                     i+=3
#             company_pre[company] = len(result)

# search_by_companies('ПАО Сбербанк')

#### Finding table with last 10 news 

In [ ]:
# def find_news():
#     url = 'https://www.e-disclosure.ru/poisk-po-soobshheniyam'

#     driver = webdriver.Firefox()
#     driver.implicitly_wait(30)
#     driver.get(url)

#     python_button = driver.find_element_by_id('butt') #FHSU
#     python_button.click() #click fhsu link
#     #  Выбрать пункт 20 и нажать
#     python_button2 = driver.find_element_by_xpath('//*[@id="pageSize"]/option[1]')
#     python_button2.click()

#     time.sleep(3)
#     soup = bs(driver.page_source, 'lxml')
#     res = soup.find_all('tbody')

#     global table
#     table = res[-9]

#     driver.quit()


### News check - frozen

In [ ]:
# def news_check(message):
#     url = 'https://www.e-disclosure.ru/'
# # set up driver for working with js - BS4 will not find the table 
#     driver = webdriver.Firefox()
#     driver.implicitly_wait(30)
#     driver.get(url)
# # press button 'search'
#     python_button = driver.find_element_by_id('butt') #FHSU
#     python_button.click() #click fhsu link
    
#     soup = bs(driver.page_source, 'lxml')
#     res = soup.find_all('a')

#     for a in range(42,72,2):
#         bot.send_message(message.chat.id, res[a].text + '\n' + str(res[a+1].text) + ' ' + str(res[a+1].get('href')))


#### Choosing only new events with the help of date-time and returning them

In [ ]:
# td_last = 240620202000
# def send_news():

#     global td_last
    
#     #  Here i need to define all the variables I will use in this module
#     def number_date(text):
#         '''Function to convert date into an int for comparison and searching relevant news'''
#         pre_res = text.split()
#         one = pre_res[0].replace('.','') 
#         two = pre_res[1].replace(':','')
#         res = int(one+two)
#         return res



#     iterat = table.contents  # all tr in the table

    
        
#     for child in range(0,len(iterat),2):
#     # for some reason there are empty cells in [0],[2],[4] and etc. rows, so I need to go through odd ones
#         date = iterat[child].contents[1].text # first td with the date and time
#         if number_date(date) > td_last:
#     #         iterat[child].contents[3] is the second td containing company name and the event name. However, there are two <a> tags, 
#     #         one with company name, the other with the event. Therefore another method, contents[0].text - company and contents[3].text - event, is used.
#             info = str(iterat[child].contents[3].contents[0].text) + '\n' + str(iterat[child].contents[3].contents[3].text) + ' ' + str(iterat[child].contents[3].contents[3].get('href'))
#             link = iterat[child].contents[3].contents[0].get('href') 


#             subscribers(link, info)

#         # td_last is the last time we checked news - it is equiualent to the top row news data in the current session.



#     td_last = number_date(iterat[0].contents[1].text) 

# Поиск и отправление новостей

In [ ]:
# td_last = 250620201900
# def find_send_news():
#     while True:
#         url = 'https://www.e-disclosure.ru/poisk-po-soobshheniyam'

#         driver = webdriver.Firefox()
#         driver.implicitly_wait(10)
#         driver.get(url)

#         python_button = driver.find_element_by_id('butt') #FHSU
#         python_button.click() #click fhsu link
#         #  Выбрать пункт 10 и нажать
#         python_button2 = driver.find_element_by_xpath('//*[@id="pageSize"]/option[1]')
#         python_button2.click()

#         time.sleep(3)
#         soup = bs(driver.page_source, 'lxml')
#         res = soup.find_all('tbody')

#         table = res[-9]

#         driver.quit()


#         global td_last

#         #  Here i need to define all the variables I will use in this module
#         def number_date(text):
#             '''Function to convert date into an int for comparison and searching relevant news'''
#             pre_res = text.split()
#             one = pre_res[0].replace('.','') 
#             two = pre_res[1].replace(':','')
#             res = int(one+two)
#             return res



#         iterat = table.contents  # all tr in the table

#         for child in range(0,len(iterat),2):
#         # for some reason there are empty cells in [0],[2],[4] and etc. rows, so I need to go through odd ones
#             date = iterat[child].contents[1].text # first td with the date and time
#             if number_date(date) > td_last:
#         #         iterat[child].contents[3] is the second td containing company name and the event name. However, there are two <a> tags, 
#         #         one with company name, the other with the event. Therefore another method, contents[0].text - company and contents[3].text - event, is used.
#                 info = str(iterat[child].contents[3].contents[0].text) + '\n' + str(iterat[child].contents[3].contents[3].text) + ' ' + str(iterat[child].contents[3].contents[3].get('href'))
#                 link = iterat[child].contents[3].contents[0].get('href') 


#                 subscribers(link, info)
#             # td_last is the last time we checked news - it is equiualent to the top row news data in the current session.



#         td_last = number_date(iterat[0].contents[1].text)   
#         time.sleep(5)